# Vaje 3: Merjenje napovedne napake modela

## Naloga 1: Prečno preverjanje in stabilnost modela

In [2]:
import numpy as np

# Preberemo podatke shranje v numpy formatu (s funkcijo numpy.save(pot, array))
data = np.load("vaje3_1.npy")
X = data[:, :-1]
y = data[:, -1]

1.a: Preveri (povprečno) točnost linearne regresije s petkratnim prečnim preverjanjem. Kako stabilen je model oz. kakšna je varianca dobljenih napak?

<details>
  <summary>Namig:</summary>

  *Pomagaj si z [objektom sklearn.model_selection.KFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) in njegovo metodo split(X)*.
   
</details>

In [22]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score


from sklearn.model_selection import KFold
kf = KFold()
n=kf.get_n_splits() # defult je petkratno prečno preverjanje

errors = []

data = kf.split(X, y)

for train_index, test_index in data:
    X_train = X[train_index,:]
    y_train = y[train_index]
    model = LinearRegression()
    model.fit(X_train, y_train)

    X_test = X[test_index,:]
    y_test = y[test_index]
    pred = model.predict(X_test)
    mse = mean_squared_error(y_test, pred)
    errors.append(mse)

mean_error = np.mean(errors)
var_error = np.var(errors)

print(f"Povprečna napaka (MSE): {mean_error:.4f}")
print(f"Varianca napak: {var_error:.6f}")


Povprečna napaka (MSE): 0.0339
Varianca napak: 0.000011


1.b: S prečnim preverjanjem sestavi pet modelov linearne regresije ter si (v matriko velikosti 5x4) shrani njihove začetne vrednosti in koeficiente. Se istoležni koeficienti v različnih vzorcih (Foldih) razlikujejo? Za koliko?

In [25]:
kf = KFold()
n=kf.get_n_splits() # defult je petkratno prečno preverjanje

koef = []
zac_vred = []

data = kf.split(X, y)

for train_index, test_index in data:
    X_train = X[train_index,:]
    y_train = y[train_index]
    model = LinearRegression()
    model.fit(X_train, y_train)

    koef.append(model.coef_)
    zac_vred.append(model.intercept_)



print('zacetne vrednosti: ', zac_vred)
print('koeficienti')
koef

zacetne vrednosti:  [np.float64(-0.18415938615951566), np.float64(-0.19639105256353862), np.float64(-0.16964681623020406), np.float64(-0.15786318095711271), np.float64(-0.18573745789261986)]
koeficienti


[array([1.50743854, 2.43282336, 1.47957335]),
 array([1.52647655, 2.4416169 , 1.47915122]),
 array([1.51169294, 2.4205532 , 1.46727743]),
 array([1.48666751, 2.43633069, 1.47178237]),
 array([1.49966098, 2.42805102, 1.50079499])]

1.c: Podatkom dodaj spremenljivke drugega reda ($x_1^2$, $x_1\cdot x_2$, $x_1\cdot x_3$, $x_2^2$, ...). Stolpce lahko združiš s funkcijo numpy.concatenate(seznam stolpcev, axis=1). 

In [53]:

x12 = (X[:, 0] **2).reshape(-1, 1)
x22 = (X[:, 1] **2).reshape(-1, 1)
x32 = (X[:, 2] **2).reshape(-1, 1)

x1x2 = (X[:, 0] * (X[:, 1])).reshape(-1, 1)
x2x3 = (X[:, 1] * (X[:, 2])).reshape(-1, 1)
x1x3 = (X[:, 0] * (X[:, 2])).reshape(-1, 1)


X_2 = np.concatenate([X, x12, x22, x32, x1x2, x2x3, x1x3], axis=1)


1.d: Preveri točnost linearne regresije na podatkih iz naloge 1.c s petkratnim prečnim preverjanjem. Se koeficienti modela bolj ali manj razlikujejo med različnimi vzorci?

In [56]:
kf = KFold()
n=kf.get_n_splits() # defult je petkratno prečno preverjanje

koef = []
zac_vred = []

data = kf.split(X_2, y)

for train_index, test_index in data:
    X_train = X_2[train_index,:]
    y_train = y[train_index]
    model = LinearRegression()
    model.fit(X_train, y_train)

    koef.append(model.coef_)
    zac_vred.append(model.intercept_)



print('zacetne vrednosti: ', zac_vred)
print('koeficienti')
koef

zacetne vrednosti:  [np.float64(-0.1569981505042164), np.float64(-0.184156344916925), np.float64(-0.15732876543555152), np.float64(-0.1461827843167276), np.float64(-0.15551723813637164)]
koeficienti


[array([ 1.79941886,  2.63834023,  1.64488464, -0.76913631, -1.01879069,
        -0.48233522,  0.95692407,  0.65824361, -0.01094716]),
 array([ 1.87320376,  2.68345029,  1.6475676 , -0.79710953, -1.05498166,
        -0.47580426,  0.94622875,  0.64410402, -0.04115949]),
 array([ 1.82902071,  2.63787618,  1.63662872, -0.75988247, -1.05697745,
        -0.46979694,  0.9632497 ,  0.68113632, -0.04824938]),
 array([ 1.79768141,  2.65642456,  1.6333356 , -0.76658894, -1.09087951,
        -0.46939182,  1.00748357,  0.68207456, -0.04180794]),
 array([ 1.7823927 ,  2.65231105,  1.67674674, -0.7542998 , -1.07344501,
        -0.5209593 ,  0.98291144,  0.6982056 , -0.02695304])]

## Naloga 2:  Stratificirano vzorčenje

In [57]:
# Preberemo podatke iz datoteke vaje3_2.npz. Podatke x, y lahko shranimo v datoteko s končnico npz z uporabo funkcije numpy.savez(pot, x=x, y=y)
data = np.load("vaje3_2.npz")
# Podatke shranimo v spremenljivko x
x = data["x"]
# Ciljne vrednosti shranimo v spremenljivko y
y = data["y"]

2.a: Preveri točnost logistične regresije s petkratnim prečnim preverjanjem. Izpiši točnost modela glede na metriko "klasifikacijska točnost" (accuracy) v vsakem vzorcu. Opaziš kaj nenavadnega?

In [62]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


# X, y = tvoji podatki
kf = KFold(n_splits=5, shuffle= False)
model = LogisticRegression(max_iter=1000)

accuracies = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)

print("Točnost v posameznem vzorcu (fold):", accuracies)
print("Povprečna točnost:", np.mean(accuracies))
print("Varianca točnosti:", np.var(accuracies))
#model je dost stabilen, razultati niso nenavadni ko je kf = KFold(n_splits=5, shuffle=True, random_state=42)
#

Točnost v posameznem vzorcu (fold): [1.0, 1.0, 1.0, 0.995, 0.0]
Povprečna točnost: 0.799
Varianca točnosti: 0.15960400000000002


Podatki se ne premešajo, ampak se razdelijo zaporedno glede na vrstni red v X in y;
če so podatki urejeni po razredu (npr. najprej vsi “0”, potem vsi “1”), bodo nekateri foldi vsebovali samo en razred.
V takem primeru model v tistih foldih:
nima podatkov o drugem razredu (npr. vidi samo “0” pri učenju),
zato se ne nauči razlikovati med razredi,
in v testnem delu, kjer nastopa drugi razred, napove vse narobe → accuracy = 0.0.

2.b: Za vsak vzorec podatkov izpiši število pozitivnih in negativnih vrednosti ciljne spremenljivke v učni in testni množici. Zakaj se je točnost modela v nalogi 2.a tako razlikovala med različnimi vzorci?

In [73]:
i=0

for train_index, test_index in kf.split(X):
    i += 1
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    print('v splitu ', i , ' je v testni množici', sum(y_test == 1), 'pozitnivnih vrednosti in ',  sum(y_test == 0), 'negativnih')




v splitu  1  je v testni množici 0 pozitnivnih vrednosti in  200 negativnih
v splitu  2  je v testni množici 0 pozitnivnih vrednosti in  200 negativnih
v splitu  3  je v testni množici 0 pozitnivnih vrednosti in  200 negativnih
v splitu  4  je v testni množici 1 pozitnivnih vrednosti in  199 negativnih
v splitu  5  je v testni množici 200 pozitnivnih vrednosti in  0 negativnih


2.c: Če je distribucija ciljne spremenljivke v vzorcih učne in testne množice podobna originalni distribuciji ciljne spremenljivke, takemu vzorčenju rečemo stratificirano vzorčenje. Sestavi stratificirane vzorce za petkratno prečno preverjanje in preveri koliko pozitivnih in koliko negativnih primerov vsebuje učna in testna množica.


<details>
  <summary>Namig:</summary>

  *Pomagaj si z [objektom sklearn.model_selection.StratifiedKFold](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html) in njegovo metodo split(x, y)*.
   
</details>

In [ ]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

i = 1
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]


    print('v splitu ', i , ' je v testni množici', sum(y_test == 1), 'pozitnivnih vrednosti in ',  sum(y_test == 0), 'negativnih')

v splitu  5  je v testni množici 40 pozitnivnih vrednosti in  160 negativnih
v splitu  5  je v testni množici 40 pozitnivnih vrednosti in  160 negativnih
v splitu  5  je v testni množici 40 pozitnivnih vrednosti in  160 negativnih
v splitu  5  je v testni množici 40 pozitnivnih vrednosti in  160 negativnih
v splitu  5  je v testni množici 41 pozitnivnih vrednosti in  159 negativnih


2.d: Preveri točnost logistične regresije s petkratnim prečnim preverjanjem na vzorcih, ki jih dobiš s stratificiranim vzorčenjem. So dobljeni modeli bolj stabilni? Si pričakoval/a da bodo dobljeni rezultati bolj stabilni?

In [76]:
kf = KFold(n_splits=5, shuffle= False)
model = LogisticRegression(max_iter=1000)

accuracies = []

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)

print("Točnost v posameznem vzorcu (fold):", accuracies)
print("Povprečna točnost:", np.mean(accuracies))
print("Varianca točnosti:", np.var(accuracies))

Točnost v posameznem vzorcu (fold): [0.8, 0.8, 0.8, 0.8, 0.795]
Povprečna točnost: 0.799
Varianca točnosti: 4.0000000000000074e-06


2.e: Pred stratificiranim vzorčenjem podatke še premešaj. To narediš tako, da objektu StratifiedKFold dodaš parameter shuffle=True. So sedaj rezultati bolj stabilni? Kaj se zgodi, če kodo poženeš večkrat?

In [86]:
kf = KFold(n_splits=5, shuffle= True)
model = LogisticRegression(max_iter=1000)

accuracies = []

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    acc = accuracy_score(y_test, y_pred)
    accuracies.append(acc)

print("Točnost v posameznem vzorcu (fold):", accuracies)
print("Povprečna točnost:", np.mean(accuracies))
print("Varianca točnosti:", np.var(accuracies))

Točnost v posameznem vzorcu (fold): [0.8, 0.8, 0.8, 0.8, 0.795]
Povprečna točnost: 0.799
Varianca točnosti: 4.0000000000000074e-06


shuffle=True poskrbi, da se vrstni red podatkov premeša, zato noben fold ne vsebuje samo začetnega ali končnega dela podatkov.

StratifiedKFold kljub temu ohrani enako razmerje med razredi v vsakem foldu.

To vodi do bolj stabilnih in reprezentativnih ocen modela.

Opomba: Parameter shuffle naredi stratificirano vzorčenje stohastično s pomočjo generatorja naključnih števil. Z metodo numpy.random.seed(celo število) lahko poskrbimo, da bo generator vedno vračal ista naključna števil in bodo posledično naši eksperimenti ponovljivi.

Pozor: Vpliv na točnost modela je le posledica random seed-a. V publikacijah se random uporablja le za ponovljivost eksperimentov (in ne kot: Najboljši rezultat dobimo pri random seed-u 18)

## Dodatna naloga

Premisli, kakšna bo povprečna točnost pri napovedovanju diskretne spremenljivke z modelom majority classifier (iz prvih vaj), če uporabimo poseben primer prečnega preverjena, ki ga imenujemo "izpusti enega" (leave-one-out). Predpostavi, da je ciljna spremenljivka enakomerno razporejena (vsaka ciljna vrednost se pojavi (število vrstic)/(število unikatnih vrednosti)-krat).